<a href="https://colab.research.google.com/github/Emtatos/Football/blob/main/AICrypto_v15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Installera nödvändiga bibliotek utan att visa varningsmeddelanden ---
!pip install --upgrade --force-reinstall yfinance ta requests pandas scikit-learn xgboost lightgbm catboost -q 2>/dev/null

# Undertryck alla Python-varningar
import warnings
warnings.filterwarnings("ignore")

# --- Importera bibliotek ---
import requests
import numpy as np
import pandas as pd
import yfinance as yf

from ta.momentum import RSIIndicator
from ta.trend import MACD, ADXIndicator
from ta.volatility import BollingerBands
from ta.volume import VolumeWeightedAveragePrice

from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# --- Telegram-konfiguration ---
TELEGRAM_BOT_TOKEN = "7205234176:AAE9OYcNsmFHm0Nc40zZ9Q-y86nsU3B8FSY"
TELEGRAM_CHAT_ID = "1968695549"

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    params = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "Markdown"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        print("✅ Signal skickad till Telegram!")
    else:
        print("⚠️ Misslyckades att skicka signal till Telegram.")

def get_bitcoin_data():
    # Hämta data med auto_adjust=True
    df = yf.download('BTC-USD', period='2y', interval='1d', auto_adjust=True)

    # Om kolumnerna är ett MultiIndex, platta till dem
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)

    # Konvertera relevanta kolumner till float
    for col in ["Close", "High", "Low", "Volume"]:
        df[col] = df[col].astype(float)

    return df

def add_indicators(df):
    # Beräkna tekniska indikatorer
    df["RSI"] = RSIIndicator(close=df["Close"], window=14).rsi()
    df["MACD"] = MACD(close=df["Close"]).macd()
    df["ADX"] = ADXIndicator(high=df["High"], low=df["Low"], close=df["Close"]).adx()
    df["BB_High"] = BollingerBands(close=df["Close"]).bollinger_hband()
    df["BB_Low"] = BollingerBands(close=df["Close"]).bollinger_lband()
    df["VWAP"] = VolumeWeightedAveragePrice(high=df["High"], low=df["Low"],
                                              close=df["Close"], volume=df["Volume"]).volume_weighted_average_price()
    # Ta bort rader med NaN-värden
    df.dropna(inplace=True)
    return df

def create_target(df):
    # Skapa målvariabel: 1 om nästa dags stängningskurs är högre än dagens, annars 0
    df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
    df.dropna(inplace=True)
    return df

def train_model(df):
    X = df[["RSI", "MACD", "ADX", "BB_High", "BB_Low", "VWAP"]]
    y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = VotingClassifier(estimators=[
        ("rf", RandomForestClassifier(n_estimators=100, random_state=42)),
        ("xgb", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)),
        ("lgbm", LGBMClassifier(random_state=42, verbose=-1)),  # tysta LightGBM
        ("cat", CatBoostClassifier(verbose=0, random_state=42))
    ], voting="soft")

    model.fit(X_train, y_train)

    # Utvärdera modellens träffsäkerhet
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"📊 Modellens träffsäkerhet: {accuracy:.2%}")

    return model, X_test

# Hämta data, lägg till indikatorer och skapa målvariabel
df = get_bitcoin_data()
if df is not None:
    df = add_indicators(df)
    df = create_target(df)

    # Träna modellen
    model, X_test = train_model(df)

    # Skapa köp/sälj-rekommendation baserat på den senaste datapunkten i testmängden
    latest_data = X_test.iloc[-1].values.reshape(1, -1)
    prediction = model.predict(latest_data)[0]

    if prediction == 1:
        recommendation = "📢 **KÖP** - AI-modellen ser en köpsignal!"
    else:
        recommendation = "📉 **SÄLJ** - AI-modellen ser en säljsignal!"

    # Skicka signalen till Telegram
    send_telegram_message(recommendation)

In [ ]:
import time

while True:
    print("⏳ Kör AICrypto_v15 och skickar signal till Telegram...")

    # Hämta data, analysera och skicka signal
    df = get_bitcoin_data()
    if df is not None:
        df = add_indicators(df)
        df = create_target(df)
        model, X_test = train_model(df)

        # Skapa köp/sälj-rekommendation baserat på den senaste datapunkten i testmängden
        latest_data = X_test.iloc[-1].values.reshape(1, -1)
        prediction = model.predict(latest_data)[0]

        if prediction == 1:
            recommendation = "📢 **KÖP** - AI-modellen ser en köpsignal!"
        else:
            recommendation = "📉 **SÄLJ** - AI-modellen ser en säljsignal!"

        send_telegram_message(recommendation)

    print("✅ Väntar i 1 timme innan nästa körning...")
    time.sleep(3600)  # Väntar 3600 sekunder (1 timme) innan den körs igen

[*********************100%***********************]  1 of 1 completed

⏳ Kör AICrypto_v15 och skickar signal till Telegram...


📊 Modellens träffsäkerhet: 51.41%
✅ Signal skickad till Telegram!
✅ Väntar i 1 timme innan nästa körning...
